In [3]:
# interpreting BERT Models with Captum library
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 

import torch 
import torch.nn as nn

from transformers import BertTokenizer, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [4]:
# define help function

In [ ]:
# define help function

In [2]:
# load model / tokenizer

In [ ]:
# predict answer (testing model)